# Constants and libraries

In [1]:
'''
In this section one defines environment variables. 
Because I used this notebook on number of machines, I implemented class especially for this. 
You may not needed in one and use just simple definitions.
'''

from system_variables import SystemVariables

# choose system according your current machine
# SYSTEM_NAME = "Windows"
# SYSTEM_NAME = "Colab"
# SYSTEM_NAME = "Kaggle"
SYSTEM_NAME = "Linux"

sv = SystemVariables(SYSTEM_NAME)
PROJECT_FOLDER = sv.get_project_folder()
SRC_FOLDER = sv.get_src_folder()
OUTPUT_FOLDER = sv.get_output_folder()
TUAB_DIRECTORY, TUAB_TRAIN, TUAB_EVAL = sv.get_TUAB_folders()
DEPR_ANON_DIRECTORY = sv.get_depr_anon_folder()
INHOUSE_DIRECTORY = sv.get_inhouse_folder()

print(SYSTEM_NAME)
print()

print(f"{PROJECT_FOLDER = }")
print(f"{SRC_FOLDER = }")
print(f"{OUTPUT_FOLDER = }")
print()

print(f"{TUAB_DIRECTORY = }")
print(f"{TUAB_TRAIN = }")
print(f"{TUAB_EVAL = }")
print()

print(f"{DEPR_ANON_DIRECTORY = }")
print()

print(f"{INHOUSE_DIRECTORY = }")
print()

Linux

PROJECT_FOLDER = ''
SRC_FOLDER = ''
OUTPUT_FOLDER = ''

TUAB_DIRECTORY = 'Data/TUAB/'
TUAB_TRAIN = 'Data/TUAB/train/normal/01_tcp_ar/'
TUAB_EVAL = 'Data/TUAB/eval/normal/01_tcp_ar/'

DEPR_ANON_DIRECTORY = 'Data/depression_anonymized/'

INHOUSE_DIRECTORY = 'Data/inhouse_dataset/EEG_baseline_with_markers_cleaned/preprocessed_data/EEG_baseline/'



In [2]:
import warnings
warnings.simplefilter("ignore")

import os
import sys
import pickle
import json
import random
from IPython.display import display, clear_output
from tqdm.auto import tqdm as tqdm_auto
import mne
import pandas as pd
from tqdm.auto import tqdm

In [3]:
sys.path.append(SRC_FOLDER)

%load_ext autoreload
%autoreload 1
%aimport utils

from utils import DEFAULT_SEED
from utils.common import check_tags, remove_files
from utils.data_reading import DataReader
from utils.plotting import dataset_hists
from utils.data_preprocessing import preprocessDepressionAnonymizedData, preprocessInhouseDatasetData, preprocessTUABdata, getTUABdataset, save_preprocessed_data

# Data preview

In [4]:
def tree(data_dir, n_files=2):
    for dir, _, filenames in os.walk(data_dir):
        print(dir)
        for filename in filenames[:n_files]:
            print(os.path.join(dir, filename))

# tree(DEPR_ANON_DIRECTORY)

# tree(INHOUSE_DIRECTORY)

# tree(TUAB_DIRECTORY)
# raw = mne.io.read_raw_edf(TUAB_TRAIN + "/aaaaaaff_s002_t000.edf", preload=False)
# print(raw.ch_names, "\nDuration:", raw.times[-1]/60, "m")
# display(raw.info)
# raw.plot(n_channels=21, duration=100, scalings=100e-6, start=20);#.set_size_inches(5, 5, forward=True)

## Inhouse dataset

In [5]:
# import pandas as pd
# from tqdm import tqdm 

# for dir in ["Health/", "MDD/"]:
#     print(len(os.listdir(INHOUSE_DIRECTORY + dir)))
#     n_secs = 0
#     for file in tqdm(os.listdir(INHOUSE_DIRECTORY + dir)):
#         n_secs += len(pd.read_csv(INHOUSE_DIRECTORY + dir + file)) / 500
#     print(n_secs/len(os.listdir(INHOUSE_DIRECTORY + dir)), n_secs)

75


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:14<00:00,  5.09it/s]


96.56 7242.0
74


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:13<00:00,  5.36it/s]

95.1891891891892 7044.0


## Depression anonymized dataset

In [14]:
for dir in ["epoch_1/", "epoch_3/"]:
    print(len(os.listdir(DEPR_ANON_DIRECTORY + dir)))
    
    # path_file = pd.read_csv(DEPR_ANON_DIRECTORY + dir + 'path_file.csv')
    # print(path_file['target'].value_counts())

    n_secs = 0
    for file in tqdm(os.listdir(DEPR_ANON_DIRECTORY + dir)):
        if "path_file" in file or 'ipynb_checkpoints' in file:
            continue
        recording = pd.read_csv(DEPR_ANON_DIRECTORY + dir + file)
        n_secs += len(recording)/125
    print(n_secs/len(os.listdir(DEPR_ANON_DIRECTORY + dir)), n_secs)

89


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:03<00:00, 27.70it/s]


60.0 5340.0
89


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:03<00:00, 28.90it/s]

60.0 5340.0


# Data preprocessing and saving

### TUAB

In [ ]:
# #preprocess
# for n_secs in [1, 2, 4, 5, 10, 15, 30, 60]:
#     print(f"Processing files for {n_secs=}")
#     # picked_channels = ['EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF']
#     picked_channels = ['EEG O1-REF', 'EEG O2-REF', 'EEG T3-REF', 'EEG T4-REF']
#     picked_channels_tag = "o1_o2_t3_t4"
#     target_freq = 128
#     n_samples_per_chunk=target_freq*n_secs
    
#     chunks_file_name = f"chunks{picked_channels_tag}_{len(picked_channels)}x{n_samples_per_chunk}"
#     targets_file_name = f"targets_{picked_channels_tag}_{len(picked_channels)}x{n_samples_per_chunk}"

#     remove_files(TUAB_TRAIN, [chunks_file_name, targets_file_name])
#     remove_files(TUAB_EVAL, [chunks_file_name, targets_file_name]) 
    
#     preprocessTUABdata(
#         picked_channels, 
#         chunks_file_name, 
#         targets_file_name, 
#         TUAB_TRAIN, 
#         TUAB_EVAL, 
    
#         target_freq=target_freq,
#         n_samples_per_chunk=n_samples_per_chunk,
        
#         force_recompute=True,
#         is_save=True,
#         n_files=None,
#         n_chunks_max=None,
#     )
    
#     dataset = getTUABdataset(
#         TUAB_TRAIN, 
#         TUAB_EVAL, 
#         chunks_file_name, 
#         targets_file_name,
        
#         n_channels=len(picked_channels),
#         n_samples_per_chunk=n_samples_per_chunk,
#         val_size=0.1,
#         SEED=SEED,
#     )
    
#     save_preprocessed_data(
#         dataset, 
#         os.path.join(TUAB_DIRECTORY, picked_channels_tag, f"dataset_{target_freq}_{n_samples_per_chunk/target_freq:.1f}.pkl")
#     )

Processing files for n_secs=1
0 files was removed from Data/TUAB/train/normal/01_tcp_ar/
0 files was removed from Data/TUAB/eval/normal/01_tcp_ar/


  0%|▎                                                                                                                               | 3/1371 [00:14<1:48:32,  4.76s/it]

File aaaaanfk_s002_t000.edf was passed, passed files: 1, read files: 3
File aaaaapfd_s001_t001.edf was passed, passed files: 2, read files: 3
File aaaaaoql_s001_t001.edf was passed, passed files: 3, read files: 3


  1%|▉                                                                                                                              | 10/1371 [00:34<1:24:43,  3.73s/it]

File aaaaapos_s001_t000.edf was passed, passed files: 4, read files: 7


...

File aaaaajqr_s002_t000.edf was passed, passed files: 419, read files: 611
File aaaaaoph_s001_t000.edf was passed, passed files: 420, read files: 611


 75%|███████████████████████████████████████████████████████████████████████████████████████████████▌                               | 1032/1371 [44:02<12:10,  2.15s/it]

File aaaaamwf_s002_t000.edf was passed, passed files: 421, read files: 612


 75%|███████████████████████████████████████████████████████████████████████████████████████████████▉                               | 1035/1371 [44:10<13:54,  2.48s/it]

File aaaaanvq_s001_t000.edf was passed, passed files: 422, read files: 614


### Depression anonymized

In [13]:
for n_secs in [1, 5, 10, 15, 30, 60]:
    epoch_folders = ["epoch_1/", "epoch_3/"]
    picked_channels = ["fz", "cz", "pz"]
    picked_channels_tag = "fz_cz_pz"
    source_freq = 125
    target_freq = 128
    n_samples_per_chunk = target_freq*n_secs
    metadata = DEPR_ANON_DIRECTORY + "metadata_anonymized.csv"
    
    chunks_list = preprocessDepressionAnonymizedData(
        DEPR_ANON_DIRECTORY, 
        epoch_folders, 
        picked_channels,
        target_freq=target_freq,
        n_samples_per_chunk=n_samples_per_chunk,
        metadata=metadata
    )
    save_preprocessed_data(chunks_list, DEPR_ANON_DIRECTORY + f"{picked_channels_tag}/dataset_{target_freq}_{n_samples_per_chunk/target_freq:.1f}.pkl")

epoch_1/


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:14<00:00,  6.34it/s]


epoch_3/


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:14<00:00,  6.31it/s]



Chunks shape: (3, 128) , chunks number: 10679
epoch_1/


...


Chunks shape: (3, 3840) , chunks number: 356
epoch_1/


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:06<00:00, 13.51it/s]


epoch_3/


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:06<00:00, 13.66it/s]


Chunks shape: (3, 7680) , chunks number: 178


### Inhouse dataset

In [13]:
for n_secs in [1, 5, 10, 15, 30, 60]:
    data_folders = ["MDD", "Health"]
    # picked_channels = ["Fz", "Cz", "Pz"]
    picked_channels = ["O1", "O2", "T3", "T4"]
    picked_channels_tag = "o1_o2_t3_t4"
    source_freq = 500
    target_freq = 128
    n_samples_per_chunk = target_freq*n_secs
    
    chunks_list = preprocessInhouseDatasetData(
        INHOUSE_DIRECTORY, 
        data_folders, 
        picked_channels,
        source_freq = source_freq,
        target_freq = target_freq,
        n_samples_per_chunk = n_samples_per_chunk
    )
    save_preprocessed_data(chunks_list, INHOUSE_DIRECTORY + f"{picked_channels_tag}/dataset_{target_freq}_{n_samples_per_chunk/target_freq:.1f}.pkl")

  0%|                                                                                                                                           | 0/149 [00:00<?, ?it/s]


KeyError: "['T3', 'T4'] not in index"

# Data load


In [ ]:
# file = DEPR_ANON_DIRECTORY + "dataset_128_10.0.pkl"
# train_set, val_set, test_set = DataReader(file, dataset_type="depression_anonymized").split()

file = TUAB_DIRECTORY + "dataset_128_1.0.pkl"
train_set, val_set, test_set = DataReader(file, dataset_type="TUAB").split()

# file = INHOUSE_DIRECTORY + "dataset_128_60.0.pkl"
# train_set, val_set, test_set = DataReader(file, dataset_type="inhouse_dataset").split()

chunks_train, chunks_val, chunks_test = train_set["chunk"], val_set["chunk"], test_set["chunk"]
targets_train, targets_val, targets_test = train_set["target"], val_set["target"], test_set["target"]

# Data exploration

In [ ]:
# dataset_hists(train_set, val_set, test_set, chunk_bins=50, chunk_range=(-5, 5))
dataset_hists(train_set, val_set, test_set, chunk_bins=50, chunk_range=(-5, 5), target_bins=10, target_range=(0, 100))